In [0]:
from pyspark.sql import types, Window, functions as F
import pandas as pd
import numpy as np
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegressionModel, LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler, PCA, PCAModel, MinMaxScaler, MinMaxScalerModel
from pyspark.mllib.evaluation import MulticlassMetrics


In [0]:
blob_container = "main-storage" # The name of your container created in https://portal.azure.com
storage_account = "team05w261" # The name of your Storage account created in https://portal.azure.com
secret_scope = "team05" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "team05-key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"
spark.conf.set(
  f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)
df = spark.read.parquet(f"{blob_url}/all_time_full_join_6")

# df_test = df.filter(F.col('YEAR_AIRLNS')==2019)
df_train = df.filter(F.col('YEAR_AIRLNS')<=2018)
# df_test = lr_predict(df_test)
# display(df_test)

In [0]:
def lr_predict(df):
    
    #df = df.withColumn('lr_id', F.monotonically_increasing_id())
    df2 = df.alias('df2')
    
    categorical_string_features = [
      'ORIGIN_AIRLNS',
      'DEST_AIRLNS',
      'OP_UNIQUE_CARRIER_AIRLNS'
    ]

    categorical_features = [
      'ORIGIN_AIRLNS_indexed',
      'DEST_AIRLNS_indexed',
      'OP_UNIQUE_CARRIER_AIRLNS_indexed',
      'HOLIDAY',
      'Prev_Flight_Delay_15',
      'Enough_Time_Btwn_Estimate_Arrival_and_Planned_Dep'
    ]

    # Run string feature indexing and one-hot encoding on the whole data set before split 
    indexer = StringIndexer(inputCols=categorical_string_features, outputCols=[col + '_indexed' for col in categorical_string_features])
    index_model = indexer.fit(df2)
    df_indexed = index_model.transform(df2).drop(*categorical_string_features)

    df_indexed = df_indexed.na.fill(value=0,subset=["Prev_Flight_Delay_15"])
    encoder = OneHotEncoder(inputCols=categorical_features, outputCols=[col + '_vec' for col in categorical_features])
    encoder_model = encoder.fit(df_indexed)
    df_encoded = encoder_model.transform(df_indexed).drop(*categorical_features)

    # Start putting data into trainable form
    actual_feature_columns = [i for i in df_encoded.columns if i != 'lr_id']
    actual_feature_columns.remove('DEP_DEL15_AIRLNS')
    actual_feature_columns.remove('FL_DATE_AIRLNS')
    vectorAssembler = VectorAssembler(inputCols = actual_feature_columns, outputCol = 'features', handleInvalid='skip')
    df_ready = vectorAssembler.transform(df_encoded).select(['features', 'DEP_DEL15_AIRLNS', 'FL_DATE_AIRLNS']).withColumnRenamed("DEP_DEL15_AIRLNS", "label")
    
    # PCA
    minMaxScaler = MinMaxScaler(inputCol='features', outputCol='features_scaled')
    scaler = minMaxScaler.fit(df_ready)
    df_ready = scaler.transform(df_ready).select('features_scaled', 'label')
    pca = PCAModel.load('dbfs:/' + 'files/shared_uploads/yizhang7210@berkeley.edu/pca_for_lr')
    df_ready = pca.transform(df_ready)
    df_ready = df_ready.select(['features_transformed', 'label']).withColumnRenamed("features_transformed", "features")

    # make preds
    model_path = 'files/shared_uploads/yizhang7210@berkeley.edu/final_logistic_regression'
    final_model_loaded = LogisticRegressionModel.load('dbfs:/' + model_path)
    df_ready = final_model_loaded.transform(df_ready)
    # rename??
    
    # Extract probabilities
    get_item=F.udf(lambda v:float(v[1]), types.FloatType())
    df_ready = df_ready.withColumn("lr_prob", get_item('probability'))
    df_ready = df_ready.withColumnRenamed('prediction', 'lr_prediction')
    df_ready = df_ready.select('lr_prob', 'lr_prediction')
    df_ready = df_ready.withColumn('lr_id', F.monotonically_increasing_id())
    
    # join preds to original dataset and return it
    df = df.withColumn('lr_id', F.monotonically_increasing_id())
    df = df.join(df_ready, on='lr_id', how='left').drop('lr_id')
    
    return df

In [0]:
print(df_train.count())

23910569

In [0]:
df_train = lr_predict(df_train)
display(df_train)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 361.0 failed 4 times, most recent failure: Lost task 7.3 in stage 361.0 (TID 3923) (10.139.64.101 executor 46): org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1209)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1215)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:442)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.writeIteratorToStream(PythonUDFRunner.scala:53)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.$anonfun$run$1(PythonRunner.scala:520)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2230)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.run(PythonRunner.scala:312)
Caused by: java.lang.IllegalArgumentException: requirement failed: The columns of A don't match the number of elements of x. A: 1114, x: 1105
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.linalg.BLAS$.gemv(BLAS.scala:551)
	at org.apache.spark.ml.linalg.Matrix.multiply(Matrices.scala:124)
	at org.apache.spark.ml.linalg.Matrix.multiply$(Matrices.scala:122)
	at org.apache.spark.ml.linalg.DenseMatrix.multiply(Matrices.scala:301)
	at org.apache.spark.ml.feature.PCAModel.$anonfun$transform$1(PCA.scala:150)
	... 18 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2828)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2775)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2769)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2769)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1305)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3036)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2977)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2965)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
Caused by: org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.Buffer

In [0]:
df2 = df_train.alias('df2')

categorical_string_features = [
  'ORIGIN_AIRLNS',
  'DEST_AIRLNS',
  'OP_UNIQUE_CARRIER_AIRLNS'
]

categorical_features = [
  'ORIGIN_AIRLNS_indexed',
  'DEST_AIRLNS_indexed',
  'OP_UNIQUE_CARRIER_AIRLNS_indexed',
  'HOLIDAY',
  'Prev_Flight_Delay_15',
  'Enough_Time_Btwn_Estimate_Arrival_and_Planned_Dep'
]

# Run string feature indexing and one-hot encoding on the whole data set before split 
indexer = StringIndexer(inputCols=categorical_string_features, outputCols=[col + '_indexed' for col in categorical_string_features])
index_model = indexer.fit(df2)
df_indexed = index_model.transform(df2).drop(*categorical_string_features)

df_indexed = df_indexed.na.fill(value=0,subset=["Prev_Flight_Delay_15"])
encoder = OneHotEncoder(inputCols=categorical_features, outputCols=[col + '_vec' for col in categorical_features])
encoder_model = encoder.fit(df_indexed)
df_encoded = encoder_model.transform(df_indexed).drop(*categorical_features)

display(df_encoded)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 484.0 failed 4 times, most recent failure: Lost task 0.3 in stage 484.0 (TID 5228) (10.139.64.56 executor 55): org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1209)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1215)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:442)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.writeIteratorToStream(PythonUDFRunner.scala:53)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.$anonfun$run$1(PythonRunner.scala:520)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2230)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.run(PythonRunner.scala:312)
Caused by: java.lang.IllegalArgumentException: requirement failed: The columns of A don't match the number of elements of x. A: 1114, x: 1105
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.linalg.BLAS$.gemv(BLAS.scala:551)
	at org.apache.spark.ml.linalg.Matrix.multiply(Matrices.scala:124)
	at org.apache.spark.ml.linalg.Matrix.multiply$(Matrices.scala:122)
	at org.apache.spark.ml.linalg.DenseMatrix.multiply(Matrices.scala:301)
	at org.apache.spark.ml.feature.PCAModel.$anonfun$transform$1(PCA.scala:150)
	... 18 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2828)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2775)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2769)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2769)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1305)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3036)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2977)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2965)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
Caused by: org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.Buffere

In [0]:
display(df_encoded)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 461.0 failed 4 times, most recent failure: Lost task 0.3 in stage 461.0 (TID 5028) (10.139.64.56 executor 55): org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1209)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1215)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:442)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.writeIteratorToStream(PythonUDFRunner.scala:53)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.$anonfun$run$1(PythonRunner.scala:520)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2230)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.run(PythonRunner.scala:312)
Caused by: java.lang.IllegalArgumentException: requirement failed: The columns of A don't match the number of elements of x. A: 1114, x: 1105
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.linalg.BLAS$.gemv(BLAS.scala:551)
	at org.apache.spark.ml.linalg.Matrix.multiply(Matrices.scala:124)
	at org.apache.spark.ml.linalg.Matrix.multiply$(Matrices.scala:122)
	at org.apache.spark.ml.linalg.DenseMatrix.multiply(Matrices.scala:301)
	at org.apache.spark.ml.feature.PCAModel.$anonfun$transform$1(PCA.scala:150)
	... 18 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2828)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2775)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2769)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2769)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1305)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3036)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2977)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2965)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
Caused by: org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.Buffere

In [0]:
# Start putting data into trainable form
actual_feature_columns = [i for i in df_encoded.columns if i not in ['DEP_DEL15_AIRLNS', 'FL_DATE_AIRLNS', 'lr_id']]
vectorAssembler = VectorAssembler(inputCols = actual_feature_columns, outputCol = 'features', handleInvalid='skip')
df_ready = vectorAssembler.transform(df_encoded).select(['features', 'DEP_DEL15_AIRLNS', 'FL_DATE_AIRLNS']).withColumnRenamed("DEP_DEL15_AIRLNS", "label")

In [0]:
display(df_ready)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 444.0 failed 4 times, most recent failure: Lost task 1.3 in stage 444.0 (TID 4820) (10.139.64.88 executor 53): org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1209)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1215)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:442)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.writeIteratorToStream(PythonUDFRunner.scala:53)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.$anonfun$run$1(PythonRunner.scala:520)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2230)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.run(PythonRunner.scala:312)
Caused by: java.lang.IllegalArgumentException: requirement failed: The columns of A don't match the number of elements of x. A: 1114, x: 1105
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.linalg.BLAS$.gemv(BLAS.scala:551)
	at org.apache.spark.ml.linalg.Matrix.multiply(Matrices.scala:124)
	at org.apache.spark.ml.linalg.Matrix.multiply$(Matrices.scala:122)
	at org.apache.spark.ml.linalg.DenseMatrix.multiply(Matrices.scala:301)
	at org.apache.spark.ml.feature.PCAModel.$anonfun$transform$1(PCA.scala:150)
	... 18 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2828)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2775)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2769)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2769)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1305)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1305)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3036)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2977)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2965)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
Caused by: org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct ,values:array >) => struct ,values:array >)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.Buffere

In [0]:
minMaxScaler = MinMaxScaler(inputCol='features', outputCol='features_scaled').fit(df_ready)
minMaxScaler.save('files/shared_uploads/yizhang7210@berkeley.edu/minmax_for_lr')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1858507102405339> in <module> 
 ----> 1 minMaxScaler = MinMaxScaler ( inputCol = 'features' , outputCol = 'features_scaled' ) . fit ( df_ready ) 
 2 minMaxScaler . save ( 'files/shared_uploads/yizhang7210@berkeley.edu/minmax_for_lr' ) 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 333 
 334 def _fit ( self , dataset ) : 
 --> 335 java_model = self . _fit_java ( dataset ) 
 336 model = self . _create_model ( java_model ) 
 337 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 330 """
 331 self . _transfer_params_to_java ( ) 
 --> 332 return self . _java_obj . fit ( dataset . _jdf ) 
 333 
 334 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o7490.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 442.0 failed 4 times, most recent failure: Lost task 3.3 in stage 442.0 (TID 4763) (10.139.64.90 executor 50): org.apache.spark.SparkException: Failed to execute user defined function(PCAModel$$Lambda$8002/913608406: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.ContextAwareIterator.hasNext(ContextAwareIterator.scala:39)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1209)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1215)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:442)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.

In [0]:
scaler = MinMaxScalerModel.load('dbfs:/' + 'files/shared_uploads/yizhang7210@berkeley.edu/minmax_for_lr')

In [0]:




# PCA
df_ready = scaler.transform(df_ready).select('features_scaled', 'label')
pca = PCAModel.load('dbfs:/' + 'files/shared_uploads/yizhang7210@berkeley.edu/pca_for_lr')
df_ready = pca.transform(df_ready)
df_ready = df_ready.select(['features_transformed', 'label']).withColumnRenamed("features_transformed", "features")

# make preds
model_path = 'files/shared_uploads/yizhang7210@berkeley.edu/final_logistic_regression'
final_model_loaded = LogisticRegressionModel.load('dbfs:/' + model_path)
df_ready = final_model_loaded.transform(df_ready)

# Extract probabilities
get_item=F.udf(lambda v:float(v[1]), types.FloatType())
df_ready = df_ready.withColumn("lr_prob", get_item('probability'))
df_ready = df_ready.withColumnRenamed('prediction', 'lr_prediction')
df_ready = df_ready.select('lr_prob', 'lr_prediction')
df_ready = df_ready.withColumn('lr_id', F.monotonically_increasing_id())

# join preds to original dataset and return it
df = df.join(df_ready, on='lr_id', how='left').drop('lr_id')

In [0]:
df_test_rdd = df_test.select('rf_prediction', 'dep_del15_airlns').withColumnRenamed('rf_prediction', 'prediction').withColumnRenamed('dep_del15_airlns', 'label').rdd
metrics = MulticlassMetrics(df_test_rdd)
    
# classification metrics
cm = metrics.confusionMatrix().toArray()
print(f'f2 score with package: {metrics.fMeasure(0.0, 2.0)}')

# confirm I'm getting the same f score here
accuracy = (cm[0][0] + cm[1][1]) / cm.sum()
precision = (cm[1][1]) / (cm[1][1] + cm[0][1])
recall = (cm[1][1]) / (cm[1][1] + cm[1][0])

def f_score(beta, precision, recall):
    return (1+beta**2) * precision * recall / (beta**2 * precision + recall)

print(f'f2 score from first principles: {f_score(2, precision, recall)}')

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


predictions = df_train\
    .withColumnRenamed('xgb_prob', 'probability')\
    .withColumnRenamed('xgb_prediction', 'rawPrediction')\
    .withColumnRenamed('DEP_DEL15_AIRLNS', 'label')\
    .select('probability', 'label', 'rawPrediction')\
    .filter(F.col('label').isNotNull())

evaluator = BinaryClassificationEvaluator(labelCol='label')

# We have only two choices: area under ROC and PR curves :-(
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))